### **Feature Engineering**

El propósito de este notebook es seleccionar, transformar o crear nuevas variables a partir de los conjuntos de datos obtenidos durante el proceso de Extracción, Transformación y Carga (ETL), con el objetivo de preparar los datos de entrada hasta que sean adecuados para su uso en el modelo de Aprendizaje Automático que se implementará posteriormente.

In [1]:
# Importando librerias
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import fastparquet as fp 
import ast
import numpy as np 
from textblob import TextBlob

#### **STEAM GAMES**

Se procede a extraer los archivos Parquet generados durante el proceso de Extracción, Transformación y Carga (ETL) con el fin de trabajar con el atributo seleccionado.

El archivo `steam_games` contiene la información sobre los juegos de la plataforma Steam:

- `genres`: El género del juego
- `app_name`: El nombre del juego.
- `release_year`: Año de lanzamiento del juego
- `price`: El precio del juego
- `item_id`: El identificador único del contenido
- `developer`: El desarrollador del juego

In [2]:
ruta_parquet = "data/steam_games.parquet"
steam_games = pd.read_parquet(ruta_parquet)
steam_games.head(3)

,item_id,app_name,genres,developer,price,release_year
0,761140,Lost Summoner Kitty,Action,Kotoshiro,4.99,2018
1,761140,Lost Summoner Kitty,Casual,Kotoshiro,4.99,2018
2,761140,Lost Summoner Kitty,Indie,Kotoshiro,4.99,2018


#### **USER REVIEWS**

El archivo `user_review` contiene información relacionada con las revisiones realizadas por los usuarios de los juegos en Steam:

- `user_id`: Es un identificador único que se asigna a cada usuario.
- `posted`: Corresponde a la fecha de publicación.
- `item_id`: Es el identificador único del juego.
- `recommend`: Señala si el usuario recomienda o no el juego.
- `review`: Contiene los comentarios y opiniones del usuario sobre el juego.


In [3]:
ruta_parquet = "data/user_reviews.parquet"
user_reviews = pd.read_parquet(ruta_parquet)
user_reviews.head(3)

,user_id,item_id,review,recommend,posted
0,76561197970982479,1250,Simple yet with great replayability. In my opi...,True,2011
1,76561197970982479,22200,It's unique and worth a playthrough.,True,2011
2,76561197970982479,43110,Great atmosphere. The gunplay can be a bit chu...,True,2011


#### **USER ITEMS**

El archivo `user_items` almacena información relacionada con la actividad de los usuarios en la plataforma, específicamente su consumo de juegos:

- `user_id`: Es un identificador único asignado a cada usuario en la plataforma.
- `item_id`: Corresponde al identificador único del elemento, en este caso, del juego.
- `item_name`: Representa el nombre del contenido consumido, es decir, el juego.
- `playtime_forever`: Indica el tiempo acumulado que un usuario ha jugado un juego en particular.

In [4]:
ruta_parquet = "data/user_items.parquet"
user_items = pd.read_parquet(ruta_parquet)
user_items.head(3)

,user_id,item_id,playtime_forever
0,76561197970982479,10,6
1,76561197970982479,30,7
2,76561197970982479,300,4733


#### **Análisis de Sentimientos**

Dentro del conjunto de datos `user_reviews`, se incluyen reseñas de juegos realizadas por diversos usuarios. La tarea consiste en crear una nueva columna llamada `sentiment_analysis`, la cual se generará mediante un análisis de sentimiento utilizando Procesamiento de Lenguaje Natural (NLP).

Para llevar a cabo este análisis de sentimientos, empleamos la biblioteca NLP (Procesamiento de Lenguaje Natural) o TextBlob, que implica la evaluación de la polaridad (positiva, negativa o neutra) de un texto, como un comentario o una reseña. La escala que aplicamos es la siguiente:

- 0: Representa un sentimiento negativo.
- 1: Indica un sentimiento neutral.
- 2: Corresponde a un sentimiento positivo.

Para la cual, tomará el valor '0' si la reseña es negativa, '1' si es neutral y '2' si es positiva. La finalidad de esta nueva columna es reemplazar la columna existente `review`, con el propósito de simplificar el trabajo de los modelos de aprendizaje automático y el análisis de datos. En los casos donde no sea posible realizar este análisis debido a la falta de reseñas escritas, la columna `sentiment_analysis` tomará el valor de '1'.

El objetivo principal de esta operación es obtener información sobre la polaridad de los comentarios o reseñas, lo que puede ser útil para análisis posteriores y modelado de datos.

In [5]:
# funcion que analisa el sentimiento con la libreria TextBlob
def sentiment_analysis(review):
    if review is None:
        return 1 #Neutral
    analisis = TextBlob(review)
    polaridad = analisis.sentiment.polarity
    if polaridad > 0.1:
        return 2 #'Positivo'
    elif polaridad < -0.1:
        return 0 #'Negativo'
    else:
        return 1 #'Neutral'

In [6]:
user_reviews ["sentiment_analisis"]= user_reviews["review"].astype(str).apply(sentiment_analysis)
user_reviews.head(5)

,user_id,item_id,review,recommend,posted,sentiment_analisis
0,76561197970982479,1250,Simple yet with great replayability. In my opi...,True,2011,2
1,76561197970982479,22200,It's unique and worth a playthrough.,True,2011,2
2,76561197970982479,43110,Great atmosphere. The gunplay can be a bit chu...,True,2011,1
3,js41637,251610,I know what you think when you see this title ...,True,2014,2
4,js41637,227300,For a simple (it's actually not all that simpl...,True,2013,1


In [7]:
#Se busca una reseña y se controla que el analisis este correcto
user_reviews["review"][4]

"For a simple (it's actually not all that simple but it can be!) truck driving Simulator, it is quite a fun and relaxing game. Playing on simple (or easy?) its just the basic WASD keys for driving but (if you want) the game can be much harder and realistic with having to manually change gears, much harder turning, etc. And reversing in this game is a ♥♥♥♥♥, as I imagine it would be with an actual truck. Luckily, you don't have to reverse park it but you get extra points if you do cause it is bloody hard. But this is suprisingly a nice truck driving game and I had a bit of fun with it."

In [8]:
user_reviews["sentiment_analisis"][4]

1

In [9]:
user_reviews["review"][1183]

'very good'

In [10]:
user_reviews["sentiment_analisis"][1183]

2

#### **DESARROLLO DE API**

Se procede a crear las funciones necesarias para que estén disponibles como endpoints en la API y puedan ser consumidas para obtener información específica sobre los datos del conjunto de datos.


#### **PlayTimeGenre**

Debe devolver año con más horas jugadas para dicho género.

Para ello se debe tener un dataframe con algunas columnas de `steam_games` y de `user_items`

In [11]:
#Se extraen las columnas para generar un nuevo dataframe y correr la función
dato1= steam_games[["item_id","genres","release_year"]]
dato2= user_items[["item_id","playtime_forever"]]
funcion1= dato2.merge(dato1, on="item_id")
funcion1= funcion1.drop("item_id", axis= 1)
funcion1= funcion1.drop_duplicates()
funcion1.reset_index(drop=True, inplace=True)
funcion1

,playtime_forever,genres,release_year
0,6,Action,2000
1,93,Action,2000
2,108,Action,2000
3,328,Action,2000
4,6275,Action,2000
...,...,...,...
722995,2353,Simulation,2009
722996,9,Education,2010
722997,212,Education,2015
722998,212,Software Training,2015


In [12]:
def PlayTimeGenre(genre: str):
    # Filtramos por genero
    data_genres = funcion1[funcion1['genres'].str.contains(genre)]
    # Agrupamos por año y sumamos las horas jugadas
    data_genres = data_genres.groupby('release_year')['playtime_forever'].sum().reset_index()
    # Obtenemos el año con mayor horas jugadas
    year = int(data_genres[data_genres['playtime_forever'] == data_genres['playtime_forever'].max()]['release_year'].values[0])    
    return {'Año de lanzamiento con más horas jugadas para Género': genre, 'Año': year}
PlayTimeGenre('Action')

{'Año de lanzamiento con más horas jugadas para Género': 'Action', 'Año': 2012}

#### **userforgenre(género: str)** 
Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

In [49]:
#Se extraen las columnas para generar un nuevo dataframe y correr la función
datos1= steam_games[["item_id","genres","release_year"]]
datos2= user_items[["item_id","user_id","playtime_forever"]]
funcion2= datos2.merge(datos1, on="item_id")
funcion2= funcion2.drop("item_id", axis= 1)
funcion2= funcion2.drop_duplicates()
funcion2.reset_index(drop=True, inplace=True)
#Cuento filas
cant_filas= len(funcion2)
#Calculo la mitad
mitad_filas= cant_filas // 10
#Selecciono la mitad superior
funcion2= funcion2.iloc[:mitad_filas]
funcion2

,user_id,playtime_forever,genres,release_year
0,76561197970982479,6,Action,2000
1,doctr,93,Action,2000
2,corrupted_soul,108,Action,2000
3,WeiEDKrSat,328,Action,2000
4,death-hunter,6275,Action,2000
...,...,...,...,...
661773,76561198045786618,96,Action,2011
661774,76561198045786618,96,Indie,2011
661775,76561198045786618,96,RPG,2011
661776,Ctoer,878,Action,2011


In [50]:
def UserForGenre(genre: str):
    # Filtramos por género
    data_genres = funcion2[funcion2['genres'].str.contains(genre)].copy()  # Copiamos el DataFrame para evitar la advertencia
    # Convertir minutos a horas y redondear a números enteros
    data_genres.loc[:, 'playtime_forever'] = (data_genres['playtime_forever'] / 60).round().astype(int)
    # Agrupamos por usuario y sumamos las horas jugadas
    data_playtime = data_genres.groupby('user_id')['playtime_forever'].sum().reset_index()
    # Obtenemos el usuario con más horas jugadas
    user = data_playtime.loc[data_playtime['playtime_forever'].idxmax()]['user_id']
    # Filtramos por usuario
    data_user = data_genres[data_genres['user_id'] == user]
    # Agrupamos por año y sumamos las horas jugadas
    data_year = data_user.groupby('release_year')['playtime_forever'].sum().reset_index()
    years = data_year.to_dict('records')
    # Obtenemos el año con más horas jugadas
    year = int(data_genres[data_genres['playtime_forever'] == data_genres['playtime_forever'].max()]['release_year'].values[0])
    
    return f"'Usuario con más horas jugadas para Género {genre}': {user}, 'Horas jugadas': {years}"

print(UserForGenre('Action'))


'Usuario con más horas jugadas para Género Action': mittensgalore, 'Horas jugadas': [{'release_year': 2000, 'playtime_forever': 1436}, {'release_year': 2003, 'playtime_forever': 8}, {'release_year': 2004, 'playtime_forever': 5121}, {'release_year': 2006, 'playtime_forever': 6}, {'release_year': 2007, 'playtime_forever': 6}, {'release_year': 2008, 'playtime_forever': 710}, {'release_year': 2009, 'playtime_forever': 4294}, {'release_year': 2010, 'playtime_forever': 177}, {'release_year': 2011, 'playtime_forever': 13}]


#### **UsersRecommend( año : int )**

Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [18]:
#Se extraen las columnas para generar un nuevo dataframe y correr la función
data1= steam_games[["item_id","app_name","release_year"]]
data2= user_reviews[["item_id","sentiment_analisis"]]
funcion3= data2.merge(data1, on="item_id")
funcion3= funcion3.drop("item_id", axis= 1)
funcion3= funcion3.drop_duplicates()
funcion3.reset_index(drop=True, inplace=True)
funcion3

,sentiment_analisis,app_name,release_year
0,2,Killing Floor,2009
1,1,Killing Floor,2009
2,0,Killing Floor,2009
3,2,Zeno Clash,2009
4,1,Zeno Clash,2009
...,...,...,...
4308,1,Millie,2014
4309,1,Aeros Quest,2015
4310,0,Another Perspective,2014
4311,2,The Howler,2016


In [19]:
def UsersRecommend( year : int ):
    # Filtramos por año
    data_year = funcion3[funcion3['release_year'] == year]
    # Verifica que exista informacion del año solicitado
    if data_year.empty:
        # Devuelve un mensaje de error
        return {f"No hay datos para el año {year}"}
    # Agrupo por juego y sumo sentimientos
    games_group = funcion3.groupby(['app_name'])['sentiment_analisis'].sum()
    # Ordeno de mayor a menor
    rank = games_group.sort_values(ascending=False)    
    # Top 3
    top_3 = rank.head(3) 
    response = []
    i = 1
    for title, j in top_3.items():
        dic = {f'Puesto {i}':title}
        response.append(dic)
        i += 1
    return {'Top 3 de juegos MÁS recomendados por usuarios para el año': year, 'Top 3': response}
UsersRecommend(2011)

{'Top 3 de juegos MÁS recomendados por usuarios para el año': 2011,
 'Top 3': [{'Puesto 1': 'Far Cry'},
  {'Puesto 2': 'Resident Evil Biohazard'},
  {'Puesto 3': 'Just Cause'}]}

#### **UsersWorstDeveloper**

Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado.

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [11]:
#Se extraen las columnas para generar un nuevo dataframe y correr la función
data01= steam_games[["item_id","developer","release_year"]]
data02= user_reviews[["item_id","sentiment_analisis","recommend"]]
funcion4= data02.merge(data01, on="item_id")
funcion4= funcion4.drop("item_id", axis= 1)
funcion4= funcion4.drop_duplicates()
funcion4.reset_index(drop=True, inplace=True)
funcion4

,sentiment_analisis,recommend,developer,release_year
0,2,True,Tripwire Interactive,2009
1,1,True,Tripwire Interactive,2009
2,0,True,Tripwire Interactive,2009
3,0,False,Tripwire Interactive,2009
4,2,False,Tripwire Interactive,2009
...,...,...,...,...
4997,1,True,Forever Entertainment S. A.,2014
4998,1,True,"Soloweb Studios,Ravens Eye Studio",2015
4999,0,False,ShaunJS,2014
5000,2,True,"Antanas Marcelionis,Renė Petrulienė",2016


In [12]:
def UsersWorstDeveloper(year: int):
    # Verificamos si el año está dentro del rango esperado
    rango_aceptado = range(2010, 2018)
    if year not in rango_aceptado:
        return {"message": "Mi base de datos solo tiene registros entre 2010 y 2017"}

    # Filtramos por comentarios no recomendados y sentiment_analysis negativo
    df_filtered = funcion4[(funcion4['recommend'] == False) & (funcion4['sentiment_analisis'] == 0)]

    # Filtramos por el año deseado
    df_filtered_year = df_filtered[df_filtered['release_year'] == year]

    # Si no hay datos para el año, retornamos mensaje
    if not df_filtered_year.empty:
        # Obtener los top 3 desarrolladores con menos recomendaciones
        top_developers = df_filtered_year['developer'].value_counts().head(3).reset_index()
        top_developers = top_developers.rename(columns={'index': 'Puesto 1', 'developer': 'Desarrollador'})

        # Modificamos la estructura del resultado
        result = [{"Puesto {}".format(i + 1): desarrollador} for i, desarrollador in enumerate(top_developers['Desarrollador'])]
    else:
        result = {"No hay juegos no recomendados para el año {}".format(year)}

    return {'Top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año': year, 'Top 3': result}
UsersWorstDeveloper(2011)

{'Top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año': 2011,
 'Top 3': [{'Puesto 1': 'CD PROJEKT RED'},
  {'Puesto 2': 'Trinoteam'},
  {'Puesto 3': 'Capcom Vancouver'}]}

#### **sentiment_analysis(año: int)**
Según el año de lanzamiento, esta función devuelve una lista que contiene la cantidad de registros de revisiones de usuarios categorizados con análisis de sentimiento.

Un ejemplo de retorno sería: {Negativo = 182, Neutral = 120, Positivo = 278}.

In [22]:
#Se extraen las columnas para generar un nuevo dataframe y correr la función
dato01= steam_games[["item_id","app_name","release_year"]]
dato02= user_reviews[["item_id","sentiment_analisis","review"]]
funcion5= dato02.merge(dato01, on="item_id")
funcion5= funcion5.drop("item_id", axis= 1)
funcion5= funcion5.drop_duplicates()
funcion5.reset_index(drop=True, inplace=True)
funcion5

,sentiment_analisis,review,app_name,release_year
0,2,Simple yet with great replayability. In my opi...,Killing Floor,2009
1,2,"Amazing, Non-stop action of blowing stuff to b...",Killing Floor,2009
2,2,"Compared to Left 4 Dead 2, this game REALLY gi...",Killing Floor,2009
3,1,Jogo ♥♥♥♥.,Killing Floor,2009
4,1,cara nas imagens esse jogo da pouco de medo ma...,Killing Floor,2009
...,...,...,...,...
39970,1,The game is a good game. I might understand th...,Aeros Quest,2015
39971,1,I can understand why Aero's Quest brings out s...,Aeros Quest,2015
39972,0,i really didn't like it. i'm sorry. slow and b...,Another Perspective,2014
39973,2,"Simple and fun, great art work.",The Howler,2016


In [23]:
def sentiment_analysis(year: int):
    # Filtramos por año
    data_year = funcion5[funcion5['release_year'] == year]
    # Agrupamos por sentimiento y contamos las reseñas
    data_year = data_year.groupby('sentiment_analisis')['review'].count().reset_index()
    # Obtenemos el top 3
    sentiment = data_year.to_dict('records')
    # Inicializar contadores
    negative_count = 0
    neutral_count = 0
    positive_count = 0
    # Contar el número de reseñas con cada sentimiento
    for s in sentiment:
        if s['sentiment_analisis'] == 0:
            negative_count += s['review']
        elif s['sentiment_analisis'] == 1:
            neutral_count += s['review']
        elif s['sentiment_analisis'] == 2:
            positive_count += s['review']
    # Crear el diccionario con los contadores
    sentiment = {'Negative': negative_count, 'Neutral': neutral_count, 'Positive': positive_count}
    return {'Según el año de lanzamiento': year, 'Sentimiento': sentiment}
sentiment_analysis(2010)

{'Según el año de lanzamiento': 2010,
 'Sentimiento': {'Negative': 216, 'Neutral': 716, 'Positive': 852}}

se elimina la columna `review` del dataframe `user_reviews`

In [24]:
user_reviews = user_reviews.drop(columns=["review"]).reset_index(drop=True)
user_reviews.columns

Index(['user_id', 'item_id', 'recommend', 'posted', 'sentiment_analisis'], dtype='object')

In [25]:
user_reviews.head()

,user_id,item_id,recommend,posted,sentiment_analisis
0,76561197970982479,1250,True,2011,2
1,76561197970982479,22200,True,2011,2
2,76561197970982479,43110,True,2011,1
3,js41637,251610,True,2014,2
4,js41637,227300,True,2013,1


#### Guardando Archivos

Se crean los archivos de dataframes en formato parquet para cada una de las funciones creadas

In [51]:
funcion1.to_parquet("data/PlayTimeGenre.parquet")
funcion2.to_parquet("data/UserForGenre.parquet")
funcion3.to_parquet("data/UsersRecommend.parquet")
funcion4.to_parquet("data/UsersWorstDeveloper.parquet")
funcion5.to_parquet("data/sentimiento_analisis.parquet")
user_reviews.to_parquet("data/reviews.parquet")